1 : Import all the libraries 

In [149]:
#basic lib
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import re
from datetime import datetime
import os
import scipy as sci
import plotly
import stat
import sklearn as sk
import warnings
import time 
import requests
warnings.filterwarnings("ignore")

from scipy.stats import skew
from scipy.stats import boxcox 

#scklit learn
import sklearn.preprocessing as pre
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.datasets import load_digits

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score
#from sklearn.metrics import mean_absolute_error, plot_confusion_matrix
from sklearn.metrics import roc_auc_score , roc_curve

from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestRegressor

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import GaussianNB

from sklearn.svm import SVC
from sklearn.svm import LinearSVC

from sklearn.multiclass import OneVsOneClassifier , OneVsRestClassifier , OutputCodeClassifier

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.neighbors import KNeighborsClassifier

from sklearn.cluster import KMeans


#plotly
import plotly
import chart_studio.plotly as py #pip install chart_studio
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot, plot
init_notebook_mode(connected=True)


# set styles and defaults
#pd.set_option('max_columns', None)
#pd.set_option('max_rows', 1000)

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png')
sns.set_context('talk')
sns.set_style('whitegrid')
#sns.set_palette('rainbow')
#sns.set(context='talk' , style = 'whitegrid', palette = 'rainbow')
#sns.set_palette("RdPu", 10)

2 : write spotify developer details . This is to be hidden . 

In [150]:
CLIENT_ID = '87c5153b7f74400ab86057e17cd8974a'
CLIENT_SECRET = 'e69632e3ef7448fd847a181730fac26c'
username = 'fv9r6gystti1tplk7pwckqi0e'
redirect_uri = 'http://localhost:3005/'
scope = 'user-read-recently-played'
#token = 'BQC5IhDvlsMO7gOSMGIWi2JO3iiwZTB-vWFl6O4dYBy1wMZJsaQXD8-YjVZjXw0uGf9RjU0gaK7cu8OBMMLTABlvjIOXAdkVoQWdDm5Ehcchh9QyBU7U1Y6N73Ty2pVlcH7I_nVR5qtHJSGwBBeESHkuBcUJDWI'


3: spotipy import and define client credentials, token

In [151]:
# spotify 
import spotipy #pip install spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

In [152]:
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=CLIENT_ID,   
                                   client_secret=CLIENT_SECRET,     
                                   redirect_uri=redirect_uri)

4: get streaming history 

In [153]:
df_a = pd.read_json("D:/ADITYA/Documents/adi new/coding/spotify/data_adi/all data/Streaming_History_Audio_2019-2020_0.json")

In [154]:
df_b = pd.read_json("D:/ADITYA/Documents/adi new/coding/spotify/data_adi/all data/Streaming_History_Audio_2020-2022_1.json")

In [155]:
df_c = pd.read_json("D:/ADITYA/Documents/adi new/coding/spotify/data_adi/all data/Streaming_History_Audio_2022-2023_2.json")

In [156]:
df = pd.concat([df_a,df_b,df_c],ignore_index=True)
df.shape

(42102, 21)

In [157]:
df['id'] = df['spotify_track_uri'].str.split(":").str[-1]
df = df.drop(['username','platform','conn_country','ip_addr_decrypted','user_agent_decrypted','incognito_mode','spotify_episode_uri',
              'episode_name','episode_show_name','reason_start','shuffle','offline','offline_timestamp','spotify_track_uri'],axis=1)
df = df.rename(columns={'master_metadata_track_name': 'trackName','master_metadata_album_artist_name':'artistName',
                        'master_metadata_album_album_name':'albumName','ts':'date_time'})


In [158]:
df.shape

(42102, 8)

find the unique elements from id column

In [159]:
df_unique = df.drop_duplicates(subset=['id'])
df_unique.shape

(7837, 8)

we have already got audio analysis from playlist songs 


In [160]:
playlist = pd.read_csv('playlist_data.csv')

In [161]:
playlist.shape

(388, 26)

In [162]:
playlist = playlist[['id','playlist']]

now we import audio analysis for these new songs 

In [163]:
new_songs = pd.read_csv('new_audio_analysis.csv')

In [164]:
new_songs = new_songs.drop(['trackName','artistName'],axis=1)

Now we merge these songs with audio analysis to our streaming history 

In [165]:
df_new = pd.merge(df,new_songs,on=['id'],how='left')
df_new.shape

(42102, 21)

In [166]:
df_new = df_new.dropna(subset=['id'])

In [167]:
df_new.shape

(41213, 21)

adding playlist names

In [168]:
df_new1 = pd.merge(df_new,playlist,on=['id'],how='left')

In [169]:
df_new1.shape 

(44004, 22)

Now we can doing some basic EDA and digging on this data

In [187]:
df_new1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43676 entries, 0 to 44003
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype                       
---  ------            --------------  -----                       
 0   date_time         43676 non-null  datetime64[ns, Asia/Kolkata]
 1   trackName         43676 non-null  object                      
 2   artistName        43676 non-null  object                      
 3   albumName         43676 non-null  object                      
 4   reason_end        43676 non-null  object                      
 5   skipped           11492 non-null  float64                     
 6   id                43676 non-null  object                      
 7   danceability      43676 non-null  float64                     
 8   loudness          43676 non-null  float64                     
 9   tempo             43676 non-null  float64                     
 10  acousticness      43676 non-null  float64                     
 11  en

lets see what the file contains 

In [185]:
df_new1.describe(include='all')

,date_time,trackName,artistName,albumName,reason_end,skipped,id,danceability,loudness,tempo,acousticness,energy,valence,liveness,speechiness,instrumentalness,key,time_signature,mode,playlist,min_played,duration,skip
count,43676,43676,43676,43676,43676,11492.000000,43676,43676.000000,43676.000000,43676.000000,43676.000000,43676.000000,43676.000000,43676.000000,43676.000000,43676.000000,43676,43676.000000,43676,43676,43676.000000,43676.000000,43676
unique,37789,7152,3273,5332,10,NaN,7746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,2,8,NaN,NaN,2
top,2021-05-26 20:59:14+05:30,Sooraj Ki Baahon Mein,The Yellow Diary,Zindagi Na Milegi Dobara,fwdbtn,NaN,723xQva0yBF7M6K3nDAG8R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,major,new songs,NaN,NaN,False
freq,36,600,1493,1200,18914,NaN,600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5943,NaN,29251,24059,NaN,NaN,27901
first,2019-02-27 08:16:51+05:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2023-11-17 02:36:41+05:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,0.441263,NaN,0.590831,0.848086,0.523515,0.383074,0.551844,0.451862,0.163030,0.081702,0.093201,NaN,3.905646,NaN,NaN,1.823353,3.839438,NaN
std,NaN,NaN,NaN,NaN,NaN,0.496560,NaN,0.161382,0.067908,0.115228,0.329532,0.211925,0.242136,0.123582,0.094534,0.236520,NaN,0.392084,NaN,NaN,1.805237,1.220220,NaN
min,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,0.233650,0.162480,0.000001,0.000532,0.000000,0.019300,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.190000,NaN
25%,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.492000,0.823983,0.451612,0.056600,0.412000,0.246000,0.094200,0.033300,0.000002,NaN,4.000000,NaN,NaN,0.050000,3.160000,NaN


In [186]:
df_new1.isna().sum()

date_time               0
trackName               0
artistName              0
albumName               0
reason_end              0
skipped             32184
id                      0
danceability            0
loudness                0
tempo                   0
acousticness            0
energy                  0
valence                 0
liveness                0
speechiness             0
instrumentalness        0
key                     0
time_signature          0
mode                    0
playlist                0
min_played              0
duration                0
skip                    0
dtype: int64

drop missing values

In [173]:
df_new1 = df_new1.dropna(subset=['key'])

Working with features to our convenience

In [174]:
replacement_mode = {1:'major',0:'minor'}
replacement_keys = {0:'C',1:'C#/Db',2:'D',3:'D#,Eb',4:'E',5:'F',6:'F#/Gb',7:'G',8:'G#/Ab',9:'A',10:'A#/Bb',11:'B',-1:'na'}
# # is sharp and b is flat 

In [175]:
df_new1['min_played'] = round(df_new1['ms_played']/60000,2)
df_new1['duration'] = round(df_new1['duration_ms']/60000,2)a
df_new1['loudness'] = (df_new1['loudness'] - (-60)) / (0 - (-60))
df_new1['tempo'] = (df_new1['tempo'] - (250)) / (0 - (250))
df_new1['key'] = df_new1['key'].replace(replacement_keys)
df_new1['mode'] = df_new1['mode'].replace(replacement_mode)


df_new1 = df_new1.drop(['ms_played','duration_ms'],axis=1)

if find out which songs were skipped without listening to the tracks 

In [176]:
df_new1['skip'] = df_new1['min_played'].apply(lambda x: x <= 1) 
# will i skip a song more probably if i have heard it just in the vicinity like say within an hr


drop songs with duration over 30 minutes since they might be podcasts

In [177]:
df_new1 = df_new1[df_new1['duration'] < 30]

convert "date_time" column to a date type ,so that we can work with it 

In [178]:
df_new1['date_time']= pd.to_datetime(df_new1['date_time']) 

as we see here , we have to change time zone to kolkata one 

In [179]:
df_new1['date_time'] = df_new1['date_time'].dt.tz_convert('Asia/Kolkata')

In [180]:
#fig = px.box(df_new,y = 'danceability',hover_name='trackName')
#fig.show()

fill nan values with new songs in playlist column 

In [181]:
df_new1['playlist'] = df_new1['playlist'].fillna('new songs')

In [182]:
file_path = 'Streaming_history_data.csv'
df_new1.to_csv(file_path,index=False)

In [183]:
df_new1[df_new1['tempo']>1].drop_duplicates(subset='id')

,date_time,trackName,artistName,albumName,reason_end,skipped,id,danceability,loudness,tempo,acousticness,energy,valence,liveness,speechiness,instrumentalness,key,time_signature,mode,playlist,min_played,duration,skip


In [184]:
#fig = px.scatter(df_new1,x = 'date_time',y = 'danceability',hover_name='trackName',color='skip')
#fig.show()

def handler(pd: "pipedream"):
  token = f'{pd.inputs["spotify"]["$auth"]["oauth_access_token"]}'
  authorization = f'Bearer {token}'
  headers = {"Authorization": authorization}

  params = {
    "type":"track",
    "q": pd.steps["create_search_string"]["$return_value"],
    "offset":pd.steps["select_offset"]["$return_value"]
  }

  r = requests.get('https://api.spotify.com/v1/search', params=params, headers=headers)
  return r.json()

get 10 random songs and then decide which playlist they get into